In [1]:
pwd

'C:\\Users\\lulu'

In [2]:
pip install -U torchmetrics

Requirement already up-to-date: torchmetrics in c:\users\lulu\anaconda3\envs\venv\lib\site-packages (1.5.2)
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install import-ipynb
import import_ipynb

In [4]:
# coding:utf-8
import torch                           #optimized tensor library for deeplearning
from torch.utils.data import Dataset   #torch.utils.data --- data loading utility. map-style(random) or iterable style; dataset ---- abstract class representing a dataset
from torch.autograd import Variable    #classes and functions implementing automatic differentiation of arbitrary scalar valued functions
import torch.nn as nn                  #basic building block for graphs
import torch.nn.functional as F        #convolution fn, pooling fn...
import torch.optim as optim            #implementing various optimization algorithm
import os                              #functions for interacting with os
import time

from torch.nn import BatchNorm1d
#provides time related functions

In [5]:
!pip install pyg-lib

ERROR: Could not find a version that satisfies the requirement pyg-lib (from versions: none)
ERROR: No matching distribution found for pyg-lib


In [6]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
from numpy import count_nonzero
from sklearn import preprocessing

In [8]:
from torch_geometric.data import Data
from torch_geometric.data.batch import Batch

In [9]:
input_size=186

In [10]:
from torchmetrics.functional.regression import r2_score

# Using Pathway Pathway Interaction

In [11]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np

# Graph Construction

In [12]:
# Load pathway-gene associations
pathway_array = np.loadtxt('kegg_pathway_interaction_186x186.txt')
pathway_cell_gene_arr_new = torch.tensor(pathway_array, dtype=torch.float32)

In [49]:
# Build pathway to gene mapping
def build_pathway_dict(pathway_array):
    pathway_dic_new = {}
    for i in range(pathway_array.shape[0]):
        genes = [j for j in range(pathway_array.shape[1]) if pathway_array[i, j] == 1]
        if genes:
            pathway_dic_new[i] = genes
    return pathway_dic_new

pathway_dic_new = build_pathway_dict(pathway_array)
num_pathways = len(pathway_dic_new)

In [50]:
# Load edge connections (symmetric matrix: interaction between pathways)
edge_array = np.loadtxt('kegg_pathway_interaction_186x186.txt')
edge_start, edge_end = [], []
for i in range(edge_array.shape[0]):
    for j in range(edge_array.shape[1]):
        if edge_array[i][j] != 0:
            edge_start.append(i)
            edge_end.append(j)

graphs = []
for i in range(num_pathways):
    gene_indices = pathway_dic_new[i]
    num_nodes = len(gene_indices)
    if num_nodes == 0:
        continue

    # Dummy placeholder features (replaced per sample)
    x = torch.randn((num_nodes, 1), dtype=torch.float32)

    # Create fully connected subgraph for now (can be updated)
    edge_index = torch.combinations(torch.arange(num_nodes), r=2).t()
    edge_index = torch.cat([edge_index, edge_index[[1, 0]]], dim=1)  # undirected

    data = Data(x=x, edge_index=edge_index)
    graphs.append(data)

# GCNN Model

In [51]:
# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim=1, hidden_dim=16, output_dim=8):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [52]:
# One model per pathway graph
gcn_models = [GCN() for _ in range(num_pathways)]
optimizers = [torch.optim.Adam(model.parameters(), lr=0.01) for model in gcn_models]

In [53]:
# Load gene expression samples
sample_df = pd.read_csv('train_set1.csv', index_col=0)

In [54]:


# Train loop: for each sample, update each graph
for epoch in range(1):  # small number of epochs for illustration
    print("epoch ",epoch)
    for sample_idx in range(sample_df.shape[0]):
        sample = sample_df.iloc[sample_idx]  # expression values

        for i in range(num_pathways):
            gene_indices = pathway_dic_new[i]
            if not gene_indices:
                continue

            # Get node features from sample expression values
            node_feats = sample.iloc[gene_indices].values.reshape(-1, 1)
            node_feats = torch.tensor(node_feats, dtype=torch.float32)

            # Update node features in graph
            graphs[i].x = node_feats

            # Forward and optimize
            model = gcn_models[i]
            optimizer = optimizers[i]

            model.train()
            optimizer.zero_grad()
            out = model(graphs[i].x, graphs[i].edge_index)

            # Dummy target: just sum node embeddings to some constant value for illustrative loss
            # target = torch.ones(out.shape[0], 8)
            # loss = F.mse_loss(out, target)
            # loss.backward()
            # optimizer.step()

print("Finished GCN training for all graphs.")


epoch  0
Finished GCN training for all graphs.


# Taking Mean to form a Graph

In [55]:
import torch

# graph_list is the list containing 186 graphs with updated node features
mean_features = []

for graph in graphs:
    # graph.x contains the node features of that graph
    node_mean = torch.mean(graph.x, dim=0)  # Mean over all nodes
    # If node features are vectors (e.g., dim > 1), you might choose to reduce further:
    # node_mean = node_mean.mean()  # To get a single scalar value
    mean_features.append(node_mean)

# Convert to tensor (optional)
mean_features_tensor = torch.stack(mean_features)  # Shape: (186, feature_dim)

# If you want just one value per graph (e.g., to get (186,) shape), reduce further
mean_scalar_features = mean_features_tensor.mean(dim=1)  # Shape: (186,)

print(mean_scalar_features.shape)  # Should be: torch.Size([186])


torch.Size([186])


In [56]:
edge_array1 = np.loadtxt('kegg_pathway_interaction_186x186.txt')

print(edge_array1)
print(edge_array1.shape)

edge_start=[]
edge_end=[]
edge_weight=[]

for i in range(edge_array1.shape[0]):    #323
    for j in range(edge_array1.shape[1]):
        if(edge_array1[i][j]!= 0):
                edge_start.append(i)
                edge_end.append(j)
                edge_weight.append(edge_array1[i][j])



print(len(edge_start))
print(len(edge_end))
print(len(edge_weight))

[[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(186, 186)
1364
1364
1364


In [57]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if GPU available run on GPU

In [58]:
# Convert edge list and weights to tensors
edge_index = torch.tensor([edge_start, edge_end], dtype=torch.long)  # shape: [2, num_edges]
edge_weight = torch.tensor(edge_weight, dtype=torch.float)

In [59]:
pathway_features = mean_scalar_features
# 2. Use the 186 pathway-level features (mean node features from individual pathway graphs)
# pathway_features = torch.rand(186, 1)  # shape: [186, 1]

if pathway_features.dim() == 1:
    pathway_features = pathway_features.unsqueeze(1)

# 3. Create the pathway graph
pathway_graph = Data(x=pathway_features, edge_index=edge_index, edge_attr=edge_weight)

print(pathway_graph)


Data(x=[186, 1], edge_index=[2, 1364], edge_attr=[1364])


In [60]:
batch_size = 128

# Passing through GCN to get array of 128 G2


In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

# === Your GCN model ===
class FinalGraphGCN(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=64, out_channels=128):
        super(FinalGraphGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        graph_embeddings = global_mean_pool(x, batch)  # Mean pooling per graph
        return graph_embeddings  # Shape: [batch_size, 128]


batched_graphs = [Data(x=pathway_graph.x.clone(),
                       edge_index=pathway_graph.edge_index.clone())
                  for _ in range(batch_size)]

# Load into a DataLoader
loader = DataLoader(batched_graphs, batch_size=128, shuffle=False)

# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FinalGraphGCN().to(device)

# === Pass through GCN ===
for batch in loader:
    batch = batch.to(device)
    output_embeddings = model(batch.x, batch.edge_index, batch.batch)
    print("Output shape:", output_embeddings.shape)  # [128, 128]


Output shape: torch.Size([128, 128])


In [ ]:

# # === Define the GCN Model ===
# class FinalGraphGCN(nn.Module):
#     def __init__(self, in_channels=1, hidden_channels=64, out_channels=128):
#         super(FinalGraphGCN, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         x = F.relu(self.conv1(x, edge_index))
#         x = self.conv2(x, edge_index)
#         graph_embedding = x.mean(dim=0)  # Mean-pool across nodes
#         return graph_embedding  # Shape: [128]


# # === Pass through GCN ===
# model = FinalGraphGCN()
# final_graph_embedding = model(pathway_graph.x, pathway_graph.edge_index)
# final_graph_embedding = final_graph_embedding.to(device)

# print("Output shape:", final_graph_embedding.shape)  # Should be: torch.Size([128])


# G1

In [25]:
# import torch
# import torch.nn as nn

# # === Simulate batch of 128 samples ===
# # Each sample is a feature vector of size 186
# batch_size = 128
# input_dim = 186
# output_dim = 128

# # Example input: batch of 128 samples, each with 186 features
# # (This would come from your precomputed mean_scalar_features for each graph in a batch)
# mean_scalar_features = torch.randn(batch_size, input_dim)  # Shape: (128, 186)

# # Move to device (if using GPU)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# mean_scalar_features = mean_scalar_features.to(device)

# # === Define FC layer: 186 → 128 ===
# fc_layer = nn.Linear(input_dim, output_dim)
# fc_layer = fc_layer.to(device)

# # === Pass through FC layer ===
# output_vectors = fc_layer(mean_scalar_features)  # Shape: (128, 128)

# print("Output shape:", output_vectors.shape)        # torch.Size([128, 128])


Output shape: torch.Size([128, 128])


In [62]:
import torch
import torch.nn as nn

# Real feature of shape (186,)
feature_186 = mean_scalar_features  # Replace this with your real computed vector

# Expand and repeat to simulate batch of 128
batch_size = 128
feature_batch = feature_186.unsqueeze(0).repeat(batch_size, 1)  # Shape: (128, 186)

# Move to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_batch = feature_batch.to(device)

# Define FC layer: 186 → 128
fc_layer = nn.Linear(186, 128).to(device)

# Pass through FC layer
output_vectors = fc_layer(feature_batch)  # Shape: (128, 128)

print("Output shape:", output_vectors.shape)  # torch.Size([128, 128])

Output shape: torch.Size([128, 128])


In [ ]:
# mean_scalar_features = mean_scalar_features.view(1, -1).float()  # Shape: (1, 186)

# # Define the fully connected layer: 186 → 128
# fc_layer = nn.Linear(186, 128)

# # Pass through the FC layer
# output_vector = fc_layer(mean_scalar_features)  # Shape: (1, 128)

# # Remove batch dim if needed
# output_vector = output_vector  # Shape: (128,)
# output_vector = output_vector.to(device)

# print(output_vector.shape)  # Should print: torch.Size([128])

# Cell line Part

In [63]:
num_cellline_genes = 4603 # 537
num_drug_genes = 4603#741
num_drug_fp=512
num_pathways = 186 #323
TINY = 1e-15
# TRAIN_NUM = 197957
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
time_start=time.time()

In [64]:

# �������ݼ�������data_loader
class DrugCellAUC(Dataset):
    """Drug Disease Interaction dataset."""
    def __init__(self, file_name, transform=None):
        self.pairs = []
        self.aucs = []


        mostline = 0
        with open(file_name, 'r') as f:
            for line in f:
                mostline += 1
                items = line.strip().split(',')
                cellLineId, drugId, auc, genes = items[0], items[1], items[2], items[3:]
                cell_drug_gene = [float(x) for x in genes[:num_cellline_genes + num_drug_genes]]
                assert len(cell_drug_gene) == num_cellline_genes + num_drug_fp
                cell_drug_gene = torch.Tensor(cell_drug_gene).cpu()
                self.pairs.append(cell_drug_gene)
                self.aucs.append(float(auc))
                # if mostline == TRAIN_NUM:
                #     break
        self.aucs = torch.FloatTensor(self.aucs).cpu()


    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        idx = idx % len(self)
        cell_drug_gene = self.pairs[idx]
        auc = self.aucs[idx]
        return cell_drug_gene, auc



In [65]:
import numpy as np

all_relations = []
with open('pathway_commongene.csv','r') as f:
    for line in f:
        _relation = line.strip().split(',')
#         print(len(_relation))
        all_relations.append(_relation)

pathway_cell_gene_arr=np.array(all_relations)
pathway_cell_gene_arr = pathway_cell_gene_arr.astype(np.uint8) # convert to int
# pathway_gene_arr = pathway_gene_arr.astype(np.float)
# print(pathway_cell_gene_arr[1,:])
print(pathway_cell_gene_arr.shape)

(186, 4603)


In [66]:
import numpy as np
# Displaying the contents of the text file
PPI_f = np.loadtxt('ppi_gte400_commongene4603x4603.txt',  usecols=range(165))
# print("\nContent in file1.txt:\n", pathway_gene[1,:])

print(PPI_f.shape)

## considered ppi score>=400 only

(4603, 165)


In [67]:
import torch
torch.cuda.empty_cache()

In [68]:
#import MaskedLinear1
#MaskedLinear1.MaskedLinear
# import cacul_r2
# cacul_r2.cacul_r
import cacul_r3
cacul_r3.cacul_r
torch.manual_seed(100)

In [69]:
ppi_edge_array_norm = PPI_f

c=0
d=0
f=0
for i in range(ppi_edge_array_norm.shape[0]):    #323

    for k in range(ppi_edge_array_norm.shape[1]):    #323
#         if(ppi_edge_array_norm[i][k]!=0):
        if(ppi_edge_array_norm[i][k]>=0.7):
            c=c+1
            if(ppi_edge_array_norm[i][k]>1):
                f=f+1
        else:
            d=d+1
print(c)
print(f)
print(d)

4956
0
754539


In [70]:
ppi_edge_start=[]
ppi_edge_end=[]
ppi_edge_weight=[]

for i in range(ppi_edge_array_norm.shape[0]):    #323
    for j in range(ppi_edge_array_norm.shape[1]):
#         if(ppi_edge_array_norm[i][j]!= 0):
        if(ppi_edge_array_norm[i][j]>=0.7):
#             if(i!=j):
                ppi_edge_start.append(i)
                ppi_edge_end.append(j)
                ppi_edge_weight.append(ppi_edge_array_norm[i][j])
#                 edge_start.append(i)
#                 edge_end.append(j)
#                 edge_weight.append(edge_array_norm[i][j])
#             else:
#                 edge_start.append(i)
#                 edge_end.append(j)
#                 edge_weight.append(edge_array_norm[i][j])



print(len(ppi_edge_start))
print(len(ppi_edge_end))
print(len(ppi_edge_weight))

4956
4956
4956


In [71]:
ppi_edge_index = torch.tensor([ppi_edge_start,
                           ppi_edge_end], dtype=torch.long)

ppi_edge_weight = torch.tensor(ppi_edge_weight, dtype=torch.float)


print(ppi_edge_index)
print(ppi_edge_weight)

tensor([[   0,    0,    1,  ..., 4597, 4597, 4602],
        [   0,   79,    1,  ...,  130,  155,   40]])
tensor([1.0000, 0.9000, 1.0000,  ..., 0.9430, 0.9380, 0.9990])


In [72]:
ppi_edge_index = torch.tensor([ppi_edge_start,
                           ppi_edge_end], dtype=torch.long).to(device)

ppi_edge_weight = torch.tensor(ppi_edge_weight, dtype=torch.float).to(device)


print(ppi_edge_index)
print(ppi_edge_weight)

tensor([[   0,    0,    1,  ..., 4597, 4597, 4602],
        [   0,   79,    1,  ...,  130,  155,   40]])
tensor([1.0000, 0.9000, 1.0000,  ..., 0.9430, 0.9380, 0.9990])


<ipython-input-72-ebd3d297f469>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ppi_edge_weight = torch.tensor(ppi_edge_weight, dtype=torch.float).to(device)


In [73]:
torch.cuda.max_split_size_mb=100000000

In [74]:
from torch.nn import LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GATConv, GATv2Conv
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_max_pool as gmp
from torch_geometric.utils import remove_self_loops
from torch.nn import LayerNorm

#MHGAT model
class MHGCNNet(torch.nn.Module):
    def __init__(self,input_size, pathway_dic_new, num_features_xd=64, num_features_xdout=1, n_output=1, num_features_xt=512,
                     embed_dim_drug=128, embed_dim_cell=128, output_dim=256, dropout=0.2, n_heads=4, n_land_genes=4603):
        super(MHGCNNet, self).__init__()
        self.input_size=input_size
        self.n_heads=n_heads
        self.output_dim=output_dim
        self.pathway_dic_new=pathway_dic_new
        self.fnn_list=[]
        self.n_land_genes=n_land_genes
        self.num_features_xdout=num_features_xdout
        #cellline layers

        self.ppi_gan1 = GCNConv(1, num_features_xdout)
        self.ppi_gan_bn1=nn.BatchNorm1d(1)

        self.ppi_gan2 = GATv2Conv(1, num_features_xdout)
        self.ppi_gan_bn2=nn.BatchNorm1d(1)

        self.celllin1 = nn.Linear(n_land_genes , 256).cpu()
        self.cellBN1=BatchNorm1d(256)
        self.celllin2 = nn.Linear(256, embed_dim_cell).cpu()
        self.cellBN2=BatchNorm1d(embed_dim_cell)

        # drug layers
        self.druglin1 = nn.Linear(num_drug_fp , 256).cpu()
        self.BN1=BatchNorm1d(256)
        self.druglin2 = nn.Linear(256, embed_dim_drug).cpu()
        self.BN2=BatchNorm1d(embed_dim_drug)

        self.celldruglin1 = nn.Linear(embed_dim_drug + embed_dim_cell + 256, 256).cpu()
        self.celldruglin2 = nn.Linear(256, 1).cpu()

        # activation and regularization
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        self.linear1 = nn.Linear(output_dim+num_features_xt,10)
        self.linear2 = nn.Linear(10,1)

        self.BN3=LayerNorm(256)
        self.batch_normGAT = nn.BatchNorm1d(n_land_genes)
        self.batch_normGAT2 = nn.BatchNorm1d(output_dim)

    def forward(self, data, output_vectors, output_embeddings):
        x, ppi_edge_index, ppi_edge_attr, drug_fp = data.x, data.ppi_edge_index, data.ppi_edge_weight, data.drug_fp
###########################################PPI GCN###################################################
        x=x.unsqueeze(2)
        x=list(x)
        data_list = [Data(x=x_, edge_index=ppi_edge_index, edge_attr=ppi_edge_attr) for x_ in x]
        batch = Batch.from_data_list(data_list)
# ##################################################################################################

        x = F.elu(self.ppi_gan_bn1(self.ppi_gan1(batch.x, batch.edge_index, batch.edge_attr)))
        x = F.dropout(x, p=0.2, training=self.training)

 ################################################################################################
        x=x.reshape(-1,self.n_land_genes*self.num_features_xdout)
        out_cell = self.cellBN1(self.celllin1(x))
        out_cell = F.relu(out_cell)
        out_cell = self.cellBN2(self.celllin2(out_cell))
        out_cell = F.relu(out_cell)


        # drug layers
        out_drug = self.BN1(self.druglin1(drug_fp))
        out_drug = F.relu(out_drug)
        out_drug = self.BN2(self.druglin2(out_drug))
        out_drug = F.relu(out_drug)


#         print("out_cell", out_cell.shape)
#         print("out_drug", out_drug.shape)
#         print("output_embeddings", output_embeddings.shape)
#         print("output_vectors", output_vectors.shape)
        
#         if out_cell.shape != (84, 128):
            # concat
#             print("if")
        out = torch.cat((out_cell, out_drug, output_vectors, output_embeddings), 1)
        z = F.dropout(self.BN3(self.celldruglin1(out)), p=0.2, training=self.training)
        z = F.relu(z)
        z=self.celldruglin2(z)


        return z

In [76]:
if __name__ == '__main__':
    print('run')
#     batch_size = 512
    batch_size = 128
    learning_rate = 0.0001#0.0001
    type = 'set'
    ID = '1'
    train_dataset = DrugCellAUC('train_set1.csv')
#     train_dataset = DrugCellAUC('/workspace/HiDRA/HiDRA_MHA/Dataset/set/train_set1.csv')
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True) #TRUE

    model = MHGCNNet(input_size, pathway_dic_new).to(device)
    criterion = nn.MSELoss()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0000001)
    #optimizer_sgd = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.0001)
    optimizer_sgd = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)

    test_dataset = DrugCellAUC('test_set1.csv')
    batch_size = 128
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True) #True
    min_test_mseloss=10.0
    it_count=0
    valloss=[]
    trloss=[]
#     min_r=''
#     min_r2=0.0


    MSE_loss = []
    model.train()

    for epoch in range(100):
        _loss = 0
        outs = []
        aucs = []
        for batch_idx, (cell_drug_genes, auc) in enumerate(train_data_loader):
            auc = Variable(auc.float()).cpu() #[512]
            batch_num = batch_idx + 1
            cell_drug_genes=np.array(cell_drug_genes.cpu())
            cell_genes=cell_drug_genes[:,:num_cellline_genes]
            drug_fp=cell_drug_genes[:,num_cellline_genes:]
            cell_genes=torch.from_numpy(cell_genes).float().to(device)
            drug_fp=torch.from_numpy(drug_fp).float().to(device)
            data = Data(x=cell_genes,  ppi_edge_index=ppi_edge_index, ppi_edge_weight=ppi_edge_weight, drug_fp=drug_fp)
            
            out = model(data, output_vectors.detach(), output_embeddings.detach())
            optimizer.zero_grad()  #uncomment
            loss = torch.sqrt(criterion(torch.squeeze(out) + TINY, auc)) #float value
            if epoch % 1 == 0:
                _out = torch.squeeze(out) #[512]
                for item in range(len(auc)):
                    outs.append(_out.data[item]) #[512]
                    aucs.append(auc.data[item]) #[512]
                _loss += loss.item()
            loss.backward(retain_graph=True)
            optimizer.step()
        if epoch % 1 == 0:
            cacul_rer = cacul_r3.cacul_r(outs,aucs)
            r ="{0:.4}".format(cacul_rer.cacul())
#             print("_loss", _loss)
            epoch_loss = "{0:.4}".format(_loss/batch_num)
            print ('epoch{},mseloss{},_pcc{}'.format(epoch,epoch_loss,r))
            train_mseloss = epoch_loss
            train_r = r
            MSE_loss.append(train_mseloss)
        with open('MSELoss_{}_{}.txt'.format(batch_size,learning_rate),'w') as f:
            for eachloss in range(len(MSE_loss)):
                f.write('{}\n'.format(MSE_loss[eachloss]))
        torch.save(model, 'gcn_{}_{}.pt'.format(batch_size, learning_rate))
        test_model = torch.load('gcn_{}_{}.pt'.format(batch_size, learning_rate), weights_only=False).cpu()
        test_model.eval()
        _loss = 0
        outs = []
        aucs = []
        for batch_idx, (cell_drug_genes, auc) in enumerate(test_data_loader):
            auc = Variable(auc.float()).cpu() #[512]
            batch_num = batch_idx + 1
            cell_drug_genes=np.array(cell_drug_genes.cpu())
            cell_genes=cell_drug_genes[:,:num_cellline_genes]
            drug_fp=cell_drug_genes[:,num_cellline_genes:]
            cell_genes=torch.from_numpy(cell_genes).float().to(device)
            drug_fp=torch.from_numpy(drug_fp).float().to(device)
            data = Data(x=cell_genes,  ppi_edge_index=ppi_edge_index, ppi_edge_weight=ppi_edge_weight, drug_fp=drug_fp)
            with torch.no_grad():
                out = test_model(data, output_vectors.detach(), output_embeddings.detach())
            _out = torch.squeeze(out)
            for item in range(len(auc)):
                outs.append(_out.data[item])
                aucs.append(auc.data[item])
            loss = torch.sqrt(criterion(torch.squeeze(out) + TINY, auc))
            _loss += loss
        cacul_rer = cacul_r3.cacul_r(outs,aucs)
        r ="{0:.4}".format(cacul_rer.cacul())
        test_r = r
        test_mseloss = "{0:.4}".format(_loss.data/batch_num)
        target = torch.tensor(aucs)
        preds = torch.tensor(outs)
        r2= r2_score(preds, target)
        print ('mseloss{},_pcc{},r2{}'.format(test_mseloss,r,r2))
        valloss.append(test_mseloss)
        time_end = time.time()
        if(float(test_mseloss)<float(min_test_mseloss)):

            torch.save(model, 'PPIENS_GCN.pt')
            min_test_mseloss=float(test_mseloss)
#             min_r=r
#             min_r2=r2
            it_count=0
        else:
            if(it_count>200):
                break
            else:
                it_count=it_count+1
print(min_test_mseloss)




run
epoch0,mseloss2.139,_pcc0.578
mseloss1.429,_pcc0.8541,r20.7178526520729065
epoch1,mseloss1.451,_pcc0.8426
mseloss1.358,_pcc0.8729,r20.7461032867431641
epoch2,mseloss1.393,_pcc0.8561
mseloss1.282,_pcc0.8802,r20.7735973000526428
epoch3,mseloss1.363,_pcc0.8626
mseloss1.275,_pcc0.8841,r20.7757683396339417
epoch4,mseloss1.349,_pcc0.8656
mseloss1.27,_pcc0.8872,r20.7777178287506104
epoch5,mseloss1.326,_pcc0.8705
mseloss1.231,_pcc0.8906,r20.7912623882293701
epoch6,mseloss1.311,_pcc0.8736
mseloss1.255,_pcc0.8936,r20.7825180888175964
epoch7,mseloss1.298,_pcc0.8762
mseloss1.217,_pcc0.8979,r20.7959712147712708
epoch8,mseloss1.282,_pcc0.8795
mseloss1.192,_pcc0.9006,r20.8039731979370117
epoch9,mseloss1.263,_pcc0.8831
mseloss1.185,_pcc0.903,r20.8064086437225342
epoch10,mseloss1.246,_pcc0.8863
mseloss1.151,_pcc0.9065,r20.8170372843742371
epoch11,mseloss1.23,_pcc0.8895
mseloss1.122,_pcc0.9094,r20.8261001110076904
epoch12,mseloss1.206,_pcc0.894
mseloss1.114,_pcc0.9104,r20.8287113904953003
epoch13,ms